In [12]:
import logging
import requests
from io import BytesIO
from pdfminer.high_level import extract_text


def download_pdf(url: str) -> BytesIO:
    """
    指定された URL から PDF をダウンロードし、BytesIO オブジェクトとして返す。
    """
    try:
        response = requests.get(url)
        response.raise_for_status()  # HTTP エラーがあれば例外を発生させる
        return BytesIO(response.content)
    except requests.exceptions.RequestException as e:
        logging.error("PDF のダウンロード中にエラーが発生しました: %s", e)
        raise


def extract_text_from_pdf(pdf_file: BytesIO) -> str:
    """
    BytesIO 形式の PDF ファイルからテキストを抽出する。
    """
    try:
        text = extract_text(pdf_file)
        return text
    except Exception as e:
        logging.error("PDF からのテキスト抽出中にエラーが発生しました: %s", e)
        raise

In [ ]:
url = "https://www.fastretailing.com/jp/ir/library/pdf/20250109_results.pdf"

# PDF をダウンロード
pdf_file = download_pdf(url)

# PDF からテキストを抽出
text = extract_text_from_pdf(pdf_file)

# 結果を標準出力に表示
print(text)

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv("../../.env")

In [4]:
import anthropic

client = anthropic.Anthropic(
    # デフォルトはos.environ.get("ANTHROPIC_API_KEY")
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
)

model_name = "claude-3-5-sonnet-latest"
# model_name = "claude-3-5-sonnet-20241022"
message = client.messages.create(
    model=model_name, max_tokens=1024, messages=[{"role": "user", "content": "Hello, Claude"}]
)
print(message.content)

In [10]:
import anthropic
import logging
import os
import requests
import tempfile
from markitdown import MarkItDown


def download_pdf(url: str, dest_path: str) -> None:
    """
    指定された URL から PDF をダウンロードし、dest_path に保存する。
    """
    try:
        response = requests.get(url)
        response.raise_for_status()  # HTTP エラーがあれば例外を発生させる
        with open(dest_path, "wb") as f:
            f.write(response.content)
    except requests.exceptions.RequestException as e:
        logging.error("PDF のダウンロード中にエラーが発生しました: %s", e)
        raise


def convert_pdf_to_markdown(pdf_path: str) -> str:
    """
    markitdown を使用して、指定された PDF ファイルから Markdown 形式のテキストを抽出する。
    """
    try:
        client = anthropic.Anthropic(
            # デフォルトはos.environ.get("ANTHROPIC_API_KEY")
            api_key=os.environ.get("ANTHROPIC_API_KEY"),
        )
        md = MarkItDown(llm_client=client, llm_model="claude-3-5-sonnet-latest")

        result = md.convert(pdf_path)
        return result.text_content
    except Exception as e:
        logging.error("PDF からの Markdown 変換中にエラーが発生しました: %s", e)
        raise

In [ ]:
# 例: URL から PDF をダウンロードし、Markdown に変換する
url = "https://www.fastretailing.com/jp/ir/library/pdf/20250109_results.pdf"
with tempfile.NamedTemporaryFile(suffix=".pdf", delete=False) as tmp:
    temp_pdf_path = tmp.name

try:
    download_pdf(url, temp_pdf_path)
    markdown_text = convert_pdf_to_markdown(temp_pdf_path)
    print(markdown_text)
finally:
    if os.path.exists(temp_pdf_path):
        os.remove(temp_pdf_path)